In [1]:
from hvo_sequence import midi_to_hvo_sequence
from hvo_sequence import ROLAND_REDUCED_MAPPING
GROOVE_MAPPING = {f'groove': [x for x in range(128)]}

import pandas as pd
import os
import numpy as np
import json

from bokeh.io import output_notebook, reset_output, show, save

try:
    reset_output()
    output_notebook()
except:
    output_notebook()

Could not import fluidsynth. AUDIO rendering will not work.


Loading BokehJS ...

In [2]:
# Collecting desired tracks
#   - Has genre
#   - Contains Guitar and Drums
#   - Constant 4/4 Time Signature

dataset_genre_drums = 'F:/Nil/Instrument2Groove2Drum/data/lmd_separated/has_genre/has_drums/'

print(len(os.listdir(dataset_genre_drums)))

11462


In [3]:
genre_annotation_json = 'F:/Nil/Instrument2Groove2Drum/data/lmd_matched_genre.json'

genre_annotation_no_duplicates = pd.read_json(genre_annotation_json).drop_duplicates(subset=['trackId'], keep='first').set_index('filename')

In [4]:
file_paths = {}

for root, dirs, files in os.walk(dataset_genre_drums):
    lakh_id = os.path.basename(root)
    if lakh_id in genre_annotation_no_duplicates.index:
        md = pd.read_csv(os.path.join(root, f"{lakh_id}.csv"))
        if (md.iloc[:, -2:] == 4).all().all():
            genre = genre_annotation_no_duplicates.loc[lakh_id]['genre']
            drums = None # Single file containing _[Drums]_
            guitars = [] # List of _[Guitar X]_ files
            for file in files:
                if "_[Drums]_" in file:
                    drums = os.path.join(root, file)
                elif "_[Guitar" in file:
                    guitars.append(os.path.join(root, file))
            if drums is not None and len(guitars):
                file_paths[lakh_id.replace(".mid", "")] = (drums, guitars, genre)

In [5]:
len(file_paths)

2746

In [6]:
def merge_hvo_sequences(hvo_sequences):
    assert all(hvo_seq.hvo.shape == hvo_sequences[0].hvo.shape for hvo_seq in hvo_sequences)

    shape = hvo_sequences[0].hvo.shape
    merged_arr = np.zeros_like(hvo_sequences[0].hvo)

    for i in range(shape[0]):
        max_val = -np.inf
        max_row = None
        for hvo_seq in hvo_sequences:
            if hvo_seq.hvo[i, 1] > max_val:
                max_val = hvo_seq.hvo[i, 1]
                max_row = hvo_seq.hvo[i, :]
        merged_arr[i, :] = max_row

    return merged_arr

In [ ]:
training_data = {}

inputs = []
outputs = []
filenames = []
genres = []

max_iter = 4

for lakh_id, values in file_paths.items():
    drum_path, guitar_paths, genre = values

    _2bar_hvo_seqs_drums = []
    _2bar_hvo_seqs_drum_grooves = []
    _2bar_hvo_seqs_guitar_grooves = []

    common_md = {
            'genre': genre,
            'lakh_id': lakh_id
        }

    # Drum HVO
    drum_hvo = midi_to_hvo_sequence(filename=drum_path, drum_mapping=ROLAND_REDUCED_MAPPING, beat_division_factors=[4])
    drum_hvo.metadata['instrument'] = 'drums'
    drum_hvo.metadata['midi_path'] = drum_path
    drum_hvo.metadata.update(common_md)
    # Adjusting lenght to be multiple of 16
    drum_n_bars = drum_hvo.number_of_steps / 16
    drum_hvo.adjust_length(int(np.round(drum_n_bars) * 16))

    # Drum Groove HVO
    drum_groove_hvo = midi_to_hvo_sequence(filename=drum_path, drum_mapping=GROOVE_MAPPING, beat_division_factors=[4])
    drum_groove_hvo.metadata['instrument'] = 'drums'
    drum_groove_hvo.metadata['midi_path'] = drum_path
    drum_groove_hvo.metadata.update(common_md)
    # Adjusting lenght to be multiple of 16
    drum_groove_hvo.adjust_length(int(np.round(drum_n_bars) * 16))

    # Guitar Grooves HVOs 
    all_guitar_grooves_hvo = []
    for guitar_path in guitar_paths:
        guitar_groove_hvo = midi_to_hvo_sequence(filename=guitar_path, drum_mapping=GROOVE_MAPPING, beat_division_factors=[4])
        guitar_groove_hvo.metadata['instrument'] = 'guitar'
        guitar_groove_hvo.metadata['midi_path'] = guitar_path # Irrelevant
        guitar_groove_hvo.metadata.update(common_md)
        # Adjusting lenght to be multiple of 16
        guitar_groove_hvo.adjust_length(int(np.round(drum_n_bars) * 16))
        all_guitar_grooves_hvo.append(guitar_groove_hvo)


    # Splitting into 2 bar segments
    for w_start in range(int(drum_n_bars)-1):
        start_step = int(w_start * 16)
        end_step = start_step + 32
        
        # Drum HVOs
        seg_drum_hvo_seq = drum_hvo.copy_empty()
        seg_drum_hvo_seq.hvo = drum_hvo.hvo[start_step:end_step]
        _2bar_hvo_seqs_drums.append(seg_drum_hvo_seq)

        # Drum Grooves HVOs
        seg_drum_groove_hvo_seq = drum_groove_hvo.copy_empty()
        seg_drum_groove_hvo_seq.hvo = drum_groove_hvo.hvo[start_step:end_step]
        _2bar_hvo_seqs_drum_grooves.append(seg_drum_groove_hvo_seq)

        # Guitar Grooves HVOs
        all_seq_guitar_hvo_seqs = []
        for guitar_groove_hvo in all_guitar_grooves_hvo:
            seg_guitar_hvo_seq = guitar_groove_hvo.copy_empty()
            seg_guitar_hvo_seq.hvo = guitar_groove_hvo.hvo[start_step:end_step]
            all_seq_guitar_hvo_seqs.append(seg_guitar_hvo_seq)
        # Merging guitar sequences into a single HVO
        seg_guitar_hvo_seq = all_seq_guitar_hvo_seqs[0].copy_empty()
        seg_guitar_hvo_seq.hvo = merge_hvo_sequences(all_seq_guitar_hvo_seqs)
        _2bar_hvo_seqs_guitar_grooves.append(seg_guitar_hvo_seq)

    training_data.update({
        lakh_id: {
            "drum_score": _2bar_hvo_seqs_drums,
            "drum_groove": _2bar_hvo_seqs_drum_grooves,
            "guitar_groove": _2bar_hvo_seqs_guitar_grooves
        }
    })

    for i in range(len(_2bar_hvo_seqs_drums)):
        inputs.append(_2bar_hvo_seqs_guitar_grooves[i].hvo)
        outputs.append(_2bar_hvo_seqs_drum_grooves[i].hvo)
        filenames.append(lakh_id)
        genres.append(genre)